In [2]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stp
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v

from datetime import datetime
from datetime import date
from datetime import timedelta

[nltk_data] Downloading package punkt to /Users/june/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/june/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Data Import

In [24]:
jsonFiles = glob('../pheme-rnr-dataset/**/source-tweet/*.json') #Can be used absolute or relative paths
print(jsonFiles)

, '../pheme-rnr-dataset/ferguson/non-rumours/498401948585181184/source-tweet/498401948585181184.json', '../pheme-rnr-dataset/ferguson/non-rumours/499679986727411715/source-tweet/499679986727411715.json', '../pheme-rnr-dataset/ferguson/non-rumours/498264393185239040/source-tweet/498264393185239040.json', '../pheme-rnr-dataset/ferguson/non-rumours/500282674200604672/source-tweet/500282674200604672.json', '../pheme-rnr-dataset/ferguson/non-rumours/499387985096237056/source-tweet/499387985096237056.json', '../pheme-rnr-dataset/ferguson/non-rumours/498293625420148736/source-tweet/498293625420148736.json', '../pheme-rnr-dataset/ferguson/non-rumours/498544167757570049/source-tweet/498544167757570049.json', '../pheme-rnr-dataset/ferguson/non-rumours/500277472068263937/source-tweet/500277472068263937.json', '../pheme-rnr-dataset/ferguson/non-rumours/500257318219837440/source-tweet/500257318219837440.json', '../pheme-rnr-dataset/ferguson/non-rumours/498487611963744256/source-tweet/49848761196374

5802

In [25]:
len(jsonFiles)

5802

In [36]:
data_list=[]
isRumorList = []

for jsonFile in jsonFiles[0:1000]:

    if jsonFile.find("non-rumours") == -1:
        isRumorList.append(1)
    else:
        isRumorList.append(0)

    with open (jsonFile, 'r') as f:
        for l in f.readlines():
            if not l.strip (): # skip empty lines
                continue
            json_data = json.loads(l)
            # print (json_data,"\n\n")
            data_list.append(json_data)
tweet_data_frame = pd.DataFrame.from_dict(data_list)

In [51]:
df_label = pd.DataFrame(isRumorList, columns=['isRumor'])
df_label

,isRumor
0,1
1,1
2,1
3,1
4,1
...,...
995,1
996,1
997,1
998,1


In [52]:
def printRumor(route):
    if route.find("non-rumours") == -1:
        print('rumor')
    else:
        print('non-rumor')

printRumor(jsonFiles[1])

rumor


In [58]:
tweet_data_frame

,contributors,truncated,text,in_reply_to_status_id,id,favorite_count,source,retweeted,coordinates,entities,...,user,geo,in_reply_to_user_id_str,possibly_sensitive,lang,created_at,in_reply_to_status_id_str,place,extended_entities,filter_level
0,None,False,EXTENDED: Dramatic video of gunfire inside hal...,None,524950899899113473,132,"<a href=""https://about.twitter.com/products/tw...",False,None,"{'symbols': [], 'user_mentions': [], 'hashtags...",...,"{'follow_request_sent': False, 'profile_use_ba...",None,None,False,en,Wed Oct 22 15:50:22 +0000 2014,None,None,{'media': [{'expanded_url': 'http://twitter.co...,NaN
1,None,False,Police have clarified that there were two shoo...,None,524990163446140928,70,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",False,None,"{'symbols': [], 'user_mentions': [], 'hashtags...",...,"{'follow_request_sent': False, 'profile_use_ba...",None,None,NaN,en,Wed Oct 22 18:26:23 +0000 2014,None,None,NaN,NaN
2,None,False,Soldier killed in Ottawa identified as Cpl. Na...,None,525039002307424256,141,"<a href=""https://about.twitter.com/products/tw...",False,None,"{'symbols': [], 'media': [{'expanded_url': 'ht...",...,"{'follow_request_sent': None, 'profile_use_bac...",None,None,False,en,Wed Oct 22 21:40:27 +0000 2014,None,None,{'media': [{'expanded_url': 'http://twitter.co...,low
3,None,False,NORAD increases number of planes on higher ale...,None,524957752138149888,39,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,None,"{'symbols': [], 'user_mentions': [], 'hashtags...",...,"{'follow_request_sent': False, 'profile_use_ba...",None,None,False,en,Wed Oct 22 16:17:36 +0000 2014,None,None,NaN,NaN
4,None,False,All 3 patients injured in #OttawaShooting rele...,None,525051365195014144,126,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,None,"{'symbols': [], 'user_mentions': [], 'hashtags...",...,"{'follow_request_sent': False, 'profile_use_ba...",None,None,NaN,en,Wed Oct 22 22:29:35 +0000 2014,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,None,False,"Flight #4U9525 initially climbed to 38,000 fee...",None,580325090367315968,202,"<a href=""https://about.twitter.com/products/tw...",False,None,"{'symbols': [], 'user_mentions': [], 'hashtags...",...,"{'follow_request_sent': False, 'profile_use_ba...",None,None,False,en,Tue Mar 24 11:07:38 +0000 2015,None,None,{'media': [{'expanded_url': 'http://twitter.co...,NaN
996,None,False,#BREAKING: A #Germanwings Airbus A320 plane ha...,None,580320945740480512,22,"<a href=""https://about.twitter.com/products/tw...",False,None,"{'user_mentions': [], 'symbols': [], 'trends':...",...,"{'follow_request_sent': None, 'profile_use_bac...",None,None,False,en,Tue Mar 24 10:51:09 +0000 2015,None,None,NaN,low
997,None,False,French officials confirm i/d of #Germanwings c...,None,581063816882065408,9,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'symbols': [], 'media': [{'expanded_url': 'ht...",...,"{'follow_request_sent': None, 'profile_use_bac...",None,None,False,en,Thu Mar 26 12:03:04 +0000 2015,None,None,{'media': [{'expanded_url': 'http://twitter.co...,low
998,None,False,French PM: fears 142-150 passengers died when ...,None,580326073776357376,34,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,None,"{'symbols': [], 'user_mentions': [{'id': 26462...",...,"{'follow_request_sent': False, 'profile_use_ba...",None,None,False,en,Tue Mar 24 11:11:32 +0000 2015,None,None,{'media': [{'expanded_url': 'http://twitter.co...,NaN


## Flatten and extract basic features

In [97]:
def flatten_tweets(tweets):

    def capitalratio(tweet_text):
        uppers = [l for l in tweet_text if l.isupper()]
        capitalratio = len(uppers) / len(tweet_text)
        return capitalratio 

    def tweets2tokens(tweet_text):

        # Tokenizing
        tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))

        # Setting url value (whether the tweet contains http link) and filter http links
        url=0
        for token in tokens:
            if token.startswith( 'http' ):
                url=1
        tokens = [token for token in tokens if not token.startswith('http')]

        ## Stemming
        # porter = PorterStemmer()
        # tokens = [porter.stem(token) for token in tokens]

        # Filtering Stop words
        # from nltk.corpus import stopwords
        # stop_words = set(stopwords.words('english'))
        # tokens = [token for token in tokens if not token in stop_words]

        return tokens,url
    
    def getposcount(tokens):
            postag = []
            poscount = {}
            poscount['Noun']=0
            poscount['Verb']=0
            poscount['Adjective'] = 0
            poscount['Pronoun']=0
            poscount['FirstPersonPronoun']=0
            poscount['SecondPersonPronoun']=0
            poscount['ThirdPersonPronoun']=0
            poscount['Adverb']=0
            Nouns = {'NN','NNS','NNP','NNPS'}
            Verbs={'VB','VBP','VBZ','VBN','VBG','VBD','To'}
            first_person_pronouns=['I','me','my','mine','we','us','our','ours']
            second_person_pronouns=['you','your','yours']
            third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']

            for word in tokens:
                w_lower=word.lower()
                if w_lower in first_person_pronouns:
                    poscount['FirstPersonPronoun']+=1
                elif w_lower in second_person_pronouns:
                    poscount['SecondPersonPronoun']+=1
                elif w_lower in third_person_pronouns:
                    poscount['ThirdPersonPronoun']+=1
            
            postag = nltk.pos_tag(tokens)
            for g1 in postag:
                if g1[1] in Nouns:
                    poscount['Noun'] += 1
                elif g1[1] in Verbs:
                    poscount['Verb']+= 1
                elif g1[1]=='ADJ'or g1[1]=='JJ':
                    poscount['Adjective']+=1
                elif g1[1]=='PRP' or g1[1]=='PRON':
                    poscount['Pronoun']+=1
                elif g1[1]=='ADV':
                    poscount['Adverb']+=1
            return poscount

    def contentlength(words):
        wordcount = len(words)
        return wordcount

    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    tweets_list = []
    total_tokens_l = []

    # Iterate through each tweet
    for tweet_obj in tweets:
        output_f = dict()

        output_f['text']= tweet_obj['text']
        output_f['text_token'], _ = tweets2tokens(tweet_obj['text'])
        total_tokens_l.extend(output_f['text_token'])

        '''POS Tagging'''
        pos_dict=getposcount(tweet_obj)
        # features.update(pos_dict)
        # print(type(pos_dict))
        # print(type(pos_dict))
        output_f.update(pos_dict)

        output_f['char_count'] = len(output_f['text'])
        output_f['word_count'] = len(output_f['text_token'])

        output_f['has_question'] = "?" in output_f["text"]
        output_f['has_exclaim'] = "!" in output_f["text"]
        output_f['has_period'] = "." in output_f["text"]
        # hasperiod=sum(c =='.' for c in tweet_text)
    
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        # output_f['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        # output_f['user-location'] = tweet_obj['user']['location']

        acc_created = datetime.strptime(tweet_obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
        tweet_created = datetime.strptime(tweet_obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
        age = (tweet_created - acc_created)
        # print(type(timedelta.total_seconds(age)))

        output_f['capital_ratio']=(capitalratio(tweet_obj['text']))

        # features=(capitalratio(data_list[0]['user']))
        output_f['tweet_count'] = np.log10(tweet_obj['user']['statuses_count'])
        output_f['listed_count'] = np.log10(tweet_obj['user']['listed_count'])
        output_f['follow_ratio'] = np.log10(tweet_obj['user']['followers_count'])
        output_f['age'] = int(timedelta.total_seconds(age)/86400)
        output_f['verified'] = tweet_obj['user']['verified']

        tweets_list.append(output_f)

    unk_tokens_l = list(set(total_tokens_l))
    print(len(total_tokens_l), len(unk_tokens_l)) # number of tokens and unique tokens

    return tweets_list

In [98]:
df = pd.DataFrame(flatten_tweets(data_list[0:2]))
df

34 29


,text,text_token,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,...,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified
0,EXTENDED: Dramatic video of gunfire inside hal...,"[extended, dramatic, video, of, gunfire, insid...",15,4,5,0,0,0,0,0,...,14,False,False,True,0.137681,4.665787,3.094820,4.764161,2071,False
1,Police have clarified that there were two shoo...,"[police, have, clarified, that, there, were, t...",14,4,4,0,0,0,0,0,...,20,False,False,True,0.050000,4.700323,3.293584,5.238716,1468,True


In [75]:
# for index, row in df.iterrows():
#     print(df.text[index],"\n")
df.loc[df['has_period']==False].head()

,text,text_token,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified
11,#OttawaShooting police say they are investigat...,"[ottawashooting, police, say, they, are, inves...",133,21,False,False,False,0.060150,2.948413,1.949390,3.230193,1671,True
12,Reports now that 2nd shooter has been shot in ...,"[reports, now, that, 2nd, shooter, has, been, ...",70,12,False,False,False,0.028571,4.277495,2.457882,4.552644,2030,False
27,CTV News confirms that Canadian authorities ha...,"[ctv, news, confirms, that, canadian, authorit...",140,20,False,False,False,0.078571,4.920108,3.723948,5.806241,2093,True
35,Ottawa Police spokesman tells @ABC News that 3...,"[ottawa, police, spokesman, tells, abc, news, ...",125,18,False,False,False,0.072000,4.986736,4.559260,6.559645,2027,True
43,FBI assisting in the case of the Ottawa shooti...,"[fbi, assisting, in, the, case, of, the, ottaw...",84,15,False,False,False,0.095238,4.700202,3.292699,5.238290,1468,True


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 2 non-null      object 
 1   text_token           2 non-null      object 
 2   Noun                 2 non-null      int64  
 3   Verb                 2 non-null      int64  
 4   Adjective            2 non-null      int64  
 5   Pronoun              2 non-null      int64  
 6   FirstPersonPronoun   2 non-null      int64  
 7   SecondPersonPronoun  2 non-null      int64  
 8   ThirdPersonPronoun   2 non-null      int64  
 9   Adverb               2 non-null      int64  
 10  char_count           2 non-null      int64  
 11  word_count           2 non-null      int64  
 12  has_question         2 non-null      bool   
 13  has_exclaim          2 non-null      bool   
 14  has_period           2 non-null      bool   
 15  capital_ratio        2 non-null      float64

In [57]:
df.join(df_label)

,text,text_token,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified,isRumor
0,EXTENDED: Dramatic video of gunfire inside hal...,"[extended, dramatic, video, of, gunfire, insid...",138,14,False,False,True,0.137681,4.665787,3.094820,4.764161,2071,False,1
1,Police have clarified that there were two shoo...,"[police, have, clarified, that, there, were, t...",120,20,False,False,True,0.050000,4.700323,3.293584,5.238716,1468,True,1
2,Soldier killed in Ottawa identified as Cpl. Na...,"[soldier, killed, in, ottawa, identified, as, ...",105,9,False,False,True,0.142857,4.665525,3.093422,4.758851,2072,False,1
3,NORAD increases number of planes on higher ale...,"[norad, increases, number, of, planes, on, hig...",140,17,False,False,True,0.085714,4.860464,4.368677,6.437026,2769,True,1
4,All 3 patients injured in #OttawaShooting rele...,"[all, 3, patients, injured, in, ottawashooting...",105,13,False,False,True,0.047619,4.408172,3.725340,5.563716,2106,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Flight #4U9525 initially climbed to 38,000 fee...","[flight, 4u9525, initially, climbed, to, 38000...",137,19,False,False,True,0.043796,3.902601,3.377306,5.330714,1801,True,1
996,#BREAKING: A #Germanwings Airbus A320 plane ha...,"[breaking, a, germanwings, airbus, a320, plane...",110,16,False,False,True,0.118182,5.187507,3.946551,5.787635,2917,True,1
997,French officials confirm i/d of #Germanwings c...,"[french, officials, confirm, id, of, germanwin...",138,18,False,False,True,0.079710,2.791691,1.477121,2.830589,1405,False,1
998,French PM: fears 142-150 passengers died when ...,"[french, pm, fears, 142150, passengers, died, ...",138,17,False,False,True,0.079710,4.502632,2.453318,3.895367,1564,False,1


# Word2vec

In [14]:
word2vec_  = w2v.Word2Vec(
    df['text_token'],
    sg = 1, 
    seed = 1,
    workers = 4,
    size = 300,
    min_count = 2,
    window = 5,
    sample = 1e-3 
)

In [15]:
print("Number of word vectors: {}".format(len(word2vec_.wv.vocab)))

Number of word vectors: 91


In [16]:
word2vec_.wv.most_similar('extended')

[('in', 0.15410608053207397),
 ('shooter', 0.14074741303920746),
 ('that', 0.13804525136947632),
 ('cirillo', 0.12065115571022034),
 ('is', 0.11968900263309479),
 ('today', 0.11726216226816177),
 ('to', 0.11567626893520355),
 ('ottnews', 0.11402125656604767),
 ('confirmed', 0.11307068169116974),
 ('national', 0.1011582538485527)]

In [22]:
word2vec_.train(df['text_token'], total_examples = word2vec_.corpus_count, epochs = word2vec_.iter)

(934, 3380)

In [37]:
word2vec_.wv.vectors.shape # vocab size / window size

(91, 300)

In [24]:
word_vectors = word2vec_.wv
vocabs = word_vectors.vocab.keys()

In [25]:
w2v_vectors = word_vectors.vectors # here you load vectors for each word in your model
w2v_indices = {word: word_vectors.vocab[word].index for word in word_vectors.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [26]:
def vectorize(line): 
    words = []
    
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
        # if word: 
            # words = np.asarray(words)
            # min_vec = words.min(axis=0)
            # max_vec = words.max(axis=0)
            # return np.concatenate((min_vec, max_vec))
    return np.asarray(words)

In [27]:
print("Tweet 1: ", df['text'][1])
print("Indice of '{}': {}".format(df['text_token'][1][0], w2v_indices[df['text_token'][1][0]]))
print("Indice of '{}': {}".format(df['text_token'][1][0], w2v_vectors[w2v_indices[df['text_token'][1][0]]]))
print("Indice of '{}': {}".format(df['text_token'][1][1], w2v_indices[df['text_token'][1][1]]))
print("Indice of '{}': {}".format(df['text_token'][1][1], w2v_vectors[w2v_indices[df['text_token'][1][1]]]))
print("\nVector of the first headline:\n", vectorize(df['text_token'][1]))

Tweet 1:  Police have clarified that there were two shootings in Ottawa today, not three: at the War Memorial and Parliament Hill.
Indice of 'police': 14
Indice of 'police': [-8.7909115e-04  2.2678173e-03  7.8279595e-04 -2.4806941e-04
  2.2085416e-04  3.7070556e-04  1.3762022e-03 -1.3553880e-03
 -2.3897479e-03  3.5416114e-03 -1.3871110e-03 -1.1577387e-03
 -1.4710624e-03 -9.4026956e-04 -3.0743168e-03  1.6168635e-03
  2.3008650e-03  1.2779868e-03  4.2597158e-04 -8.8723429e-04
 -1.4630724e-03 -2.5113446e-03  1.1259553e-03  4.6747914e-04
  1.1300911e-03  1.2301856e-03 -1.2631878e-03 -1.3203674e-03
 -1.1576045e-03  1.6581834e-03  2.7205111e-03  4.7297205e-04
 -2.2558081e-03 -6.5106875e-04  9.0882956e-04  3.4149017e-03
 -2.4374521e-04 -7.7263417e-04 -2.5090321e-03  9.2527538e-04
 -1.3953398e-04  2.2183282e-03  2.5357136e-03  1.1761730e-03
 -9.0547965e-04  7.3233241e-04 -8.4784382e-04 -1.0688273e-03
 -8.5110922e-04 -1.6235624e-03  2.2432089e-03  2.1959362e-03
 -2.6698710e-04  5.3775246e-04  1

In [38]:
import copy
df['text_token_vec'] = copy.deepcopy(df['text_token'])

for index, sentence in enumerate(df['text_token_vec']):
    df['text_token_vec'][index] = vectorize(sentence).mean(axis=0)
    # df['text_token_vec'][0].mean(axis=0)

print(len(df['text_token_vec'][0]))
df[['text_token','text_token_vec']].head()

300


,text_token,text_token_vec
0,"[extended, dramatic, video, of, gunfire, insid...","[-0.00036248728, 0.003031671, -3.3325305e-05, ..."
1,"[police, have, clarified, that, there, were, t...","[-0.00023865417, 0.0041637407, -0.00036229126,..."
2,"[soldier, killed, in, ottawa, identified, as, ...","[0.00019319016, 0.003343624, -0.0006239494, 2...."
3,"[norad, increases, number, of, planes, on, hig...","[-0.0012566921, 0.0022387316, 0.00034883057, 0..."
4,"[all, 3, patients, injured, in, ottawashooting...","[-9.845568e-05, 0.003184773, -0.0003556437, 0...."


In [39]:
df_avg = pd.DataFrame(df['text_token_vec'].values.tolist()).add_prefix('token_avg')#.join(df)

In [40]:
df_avg.head()

,token_avg0,token_avg1,token_avg2,token_avg3,token_avg4,token_avg5,token_avg6,token_avg7,token_avg8,token_avg9,...,token_avg290,token_avg291,token_avg292,token_avg293,token_avg294,token_avg295,token_avg296,token_avg297,token_avg298,token_avg299
0,-0.000362,0.003032,-0.000033,0.000764,0.000562,0.002019,0.000485,0.000402,-0.001589,0.002466,...,-0.000710,0.001461,-0.001360,-0.000839,0.001078,-0.000220,0.000509,0.000607,-0.000904,-0.002605
1,-0.000239,0.004164,-0.000362,0.000563,0.000864,0.003096,0.000252,0.000395,-0.001798,0.003777,...,-0.000787,0.001626,-0.001098,-0.000841,0.001279,-0.000745,0.000655,0.000847,-0.001636,-0.002769
2,0.000193,0.003344,-0.000624,0.000021,0.000610,0.002801,0.000233,0.000530,-0.001825,0.002633,...,-0.000589,0.001662,-0.001044,-0.001242,0.000846,-0.000242,0.000367,0.000623,-0.001472,-0.003005
3,-0.001257,0.002239,0.000349,0.000763,0.000656,0.002196,-0.000204,0.000054,-0.001752,0.002196,...,-0.000796,0.000938,-0.001104,-0.001364,0.001031,-0.000012,-0.000142,0.000074,-0.001242,-0.002073
4,-0.000098,0.003185,-0.000356,0.000714,0.000728,0.002770,-0.000517,0.000390,-0.001722,0.002985,...,-0.000314,0.001454,-0.001179,-0.000891,0.001048,-0.000578,0.000132,-0.000038,-0.001487,-0.002678


### Average of Vectors & Previous features

In [32]:
df.join(df_avg).head()

,text,text_token,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,...,token_avg290,token_avg291,token_avg292,token_avg293,token_avg294,token_avg295,token_avg296,token_avg297,token_avg298,token_avg299
0,EXTENDED: Dramatic video of gunfire inside hal...,"[extended, dramatic, video, of, gunfire, insid...",138,14,False,False,True,0.137681,4.665787,3.094820,...,-0.000710,0.001461,-0.001360,-0.000839,0.001078,-0.000220,0.000509,0.000607,-0.000904,-0.002605
1,Police have clarified that there were two shoo...,"[police, have, clarified, that, there, were, t...",120,20,False,False,True,0.050000,4.700323,3.293584,...,-0.000787,0.001626,-0.001098,-0.000841,0.001279,-0.000745,0.000655,0.000847,-0.001636,-0.002769
2,Soldier killed in Ottawa identified as Cpl. Na...,"[soldier, killed, in, ottawa, identified, as, ...",105,9,False,False,True,0.142857,4.665525,3.093422,...,-0.000589,0.001662,-0.001044,-0.001242,0.000846,-0.000242,0.000367,0.000623,-0.001472,-0.003005
3,NORAD increases number of planes on higher ale...,"[norad, increases, number, of, planes, on, hig...",140,17,False,False,True,0.085714,4.860464,4.368677,...,-0.000796,0.000938,-0.001104,-0.001364,0.001031,-0.000012,-0.000142,0.000074,-0.001242,-0.002073
4,All 3 patients injured in #OttawaShooting rele...,"[all, 3, patients, injured, in, ottawashooting...",105,13,False,False,True,0.047619,4.408172,3.725340,...,-0.000314,0.001454,-0.001179,-0.000891,0.001048,-0.000578,0.000132,-0.000038,-0.001487,-0.002678


# Doc2Vec

In [42]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [43]:
#Doc2vec 실행
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['text_token'])]
model = Doc2Vec(documents, vector_size=300, min_alpha=0.025, window=10, min_count=1, workers=4)

In [44]:
# document vector의 정보
len(model.docvecs[0])

300

In [46]:
def most_similar_text(index):
    similar = model.docvecs.most_similar(index)

    print("The quried text: \n\n{} \n\nis most similar to the text:\n\n{}".format(df['text'][index],df['text'][similar[0][0]]))

In [45]:
model.docvecs.most_similar(1)

[(31, 0.13479533791542053),
 (12, 0.11979718506336212),
 (4, 0.09530271589756012),
 (34, 0.0881328135728836),
 (5, 0.08271204680204391),
 (15, 0.07966940850019455),
 (8, 0.07744815945625305),
 (47, 0.07733796536922455),
 (19, 0.0739109143614769),
 (44, 0.07232490181922913)]

In [47]:
most_similar_text(30)

The quried text: 

Senior U.S. official: Canadian government has informed U.S. that one shooter is dead in Ottawa. Live blog: http://t.co/q98AMohu7T 

is most similar to the text:

Twenty-four-year-old Cpl. Nathan Cirillo was shot dead today at the War Memorial in #Ottawa. #cdnpoli #OttawaShooting http://t.co/j42xy9LX5k


## Convert to feature for df

In [48]:
list_doc = []
for index, tokens in enumerate(df['text_token']):
    list_doc.append(model.docvecs[index])

In [52]:
df_doc = pd.DataFrame(list_doc).add_prefix('doc_vec')

In [53]:
df_doc.head()

,doc_vec0,doc_vec1,doc_vec2,doc_vec3,doc_vec4,doc_vec5,doc_vec6,doc_vec7,doc_vec8,doc_vec9,...,doc_vec290,doc_vec291,doc_vec292,doc_vec293,doc_vec294,doc_vec295,doc_vec296,doc_vec297,doc_vec298,doc_vec299
0,0.001254,0.000135,0.000481,0.000981,-0.000149,0.000084,0.000492,-0.000723,0.000023,0.000389,...,-0.000729,-0.001183,-0.000241,0.000246,-0.000162,-0.001003,0.000154,-0.000411,0.001321,0.001406
1,0.001397,-0.000555,-0.000488,-0.000673,-0.001624,0.000628,0.000476,0.001602,-0.000383,-0.000133,...,0.000500,0.000647,-0.000827,0.001236,-0.001421,0.001280,0.000317,0.001384,0.000516,-0.000400
2,-0.000193,-0.000940,-0.000979,-0.000738,0.001395,-0.001106,0.000247,-0.000483,-0.000823,0.001396,...,-0.000135,0.000966,0.000596,0.001133,-0.001446,-0.000983,0.000215,-0.000484,-0.001110,-0.000691
3,-0.001311,-0.000474,-0.001242,-0.000106,-0.000916,0.001561,0.000604,-0.000501,-0.000529,0.000829,...,0.000454,-0.001348,-0.001652,-0.000885,0.001322,-0.000220,-0.000741,-0.001046,-0.000123,0.001043
4,-0.001080,-0.001007,0.001325,0.001059,0.000241,0.001430,-0.000641,0.001540,0.001282,-0.000097,...,0.001457,0.000818,0.000454,0.001714,-0.000894,0.001281,-0.000675,-0.001235,0.000623,-0.001579


In [55]:
df.join(df_doc).head()

,text,text_token,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,...,doc_vec290,doc_vec291,doc_vec292,doc_vec293,doc_vec294,doc_vec295,doc_vec296,doc_vec297,doc_vec298,doc_vec299
0,EXTENDED: Dramatic video of gunfire inside hal...,"[extended, dramatic, video, of, gunfire, insid...",138,14,False,False,True,0.137681,4.665787,3.094820,...,-0.000729,-0.001183,-0.000241,0.000246,-0.000162,-0.001003,0.000154,-0.000411,0.001321,0.001406
1,Police have clarified that there were two shoo...,"[police, have, clarified, that, there, were, t...",120,20,False,False,True,0.050000,4.700323,3.293584,...,0.000500,0.000647,-0.000827,0.001236,-0.001421,0.001280,0.000317,0.001384,0.000516,-0.000400
2,Soldier killed in Ottawa identified as Cpl. Na...,"[soldier, killed, in, ottawa, identified, as, ...",105,9,False,False,True,0.142857,4.665525,3.093422,...,-0.000135,0.000966,0.000596,0.001133,-0.001446,-0.000983,0.000215,-0.000484,-0.001110,-0.000691
3,NORAD increases number of planes on higher ale...,"[norad, increases, number, of, planes, on, hig...",140,17,False,False,True,0.085714,4.860464,4.368677,...,0.000454,-0.001348,-0.001652,-0.000885,0.001322,-0.000220,-0.000741,-0.001046,-0.000123,0.001043
4,All 3 patients injured in #OttawaShooting rele...,"[all, 3, patients, injured, in, ottawashooting...",105,13,False,False,True,0.047619,4.408172,3.725340,...,0.001457,0.000818,0.000454,0.001714,-0.000894,0.001281,-0.000675,-0.001235,0.000623,-0.001579


# POS tagging

NameError: name 'df' is not defined